In [1]:
import logging
logging.basicConfig(
    level=logging.INFO,
    format='[%(levelname)s] %(name)s - %(message)s'
)
sensitive_attributes = {'sex' : {'columns' : [
                                            {
                                            "name": "sex",
                                            "underprivileged": [2]
                                            }
                                        ],
                             'type' : 'simple'},
                        'ethnicity' : {'columns' : [
                                                                {
                                                                "name": "ethnicity",
                                                                "privileged": [1]
                                                                }
                                                            ],
                                                'type' : 'simple'},
                        'age' : {'columns' : [
                                            {
                                            "name": "age",
                                            "privileged": [3,4]
                                            }
                                        ],
                             'type' : 'simple'},
                        'sex_ethnicity' : {'groups' : ["sex","ethnicity"],
                                                        'type' : 'complex'}}


from eticas.model.ml_model import MLModel
model = MLModel(
    model_name="ML Testing Regression",
    description="A logistic regression model to illustrate audits",
    country="USA",
    state="CA",
    sensitive_attributes=sensitive_attributes,
    features=["feature_0", "feature_1", "feature_2"]
)


model.run_training_audit(dataset_path ='files/example_training_binary_2.csv', 
                                                label_column = 'outcome', output_column = 'predicted_outcome',positive_output = [1])
model.run_production_audit(dataset_path ='files/example_operational_binary_2.csv', 
                                                output_column = 'predicted_outcome',positive_output = [1])
model.run_impacted_audit(dataset_path ='files/example_impact_binary_2.csv', 
                                                output_column = 'recorded_outcome',positive_output = [1])
model.run_drift_audit(dataset_path_dev = 'files/example_training_binary_2.csv', 
                      output_column_dev = 'outcome', positive_output_dev = [1],
                    dataset_path_prod = 'files/example_operational_binary_2.csv', 
                      output_column_prod = 'predicted_outcome', positive_output_prod = [1])
audit_result = model.df_results(norm_values=True)

[INFO] numexpr.utils - NumExpr defaulting to 8 threads.
[INFO] eticas.model.ml_model - Running training audit for model: ML Testing Regression
[INFO] eticas.audit.labeled_audit - Running training audit on model 'ML Testing Regression'
[INFO] eticas.data.loaders - Dataset successfully loaded from files/example_training_binary_2.csv (CSV).
[INFO] eticas.audit.labeled_audit - Training data loaded '(10000, 14)'
[INFO] eticas.metrics.da_inconsistency - Completed: 'Da_inconsistency'
[INFO] eticas.metrics.da_positive - Completed: 'Da_positive'
[INFO] eticas.metrics.dxa_inconsistency - Completed: 'Dxa_inconsistency'
[INFO] eticas.metrics.da_informative - Completed: 'Da_informative'
[INFO] eticas.metrics.d_statisticalparity - Completed: 'D_statisticalparity'
[INFO] eticas.metrics.d_parity - Completed: 'D_parity'
[INFO] eticas.metrics.performance - Completed: 'Performance'
[INFO] eticas.metrics.d_equalodds - Completed: 'D_equalodds'
[INFO] eticas.metrics.d_calibrated - Completed: 'D_calibrated'


In [2]:
audit_result

value
group        metric           attribute     stage            
benchmarking da_inconsistency age           impact       45.0
                                            production   44.5
                                            training     44.8
                              ethnicity     impact       10.0
                                            production   20.0
...                                                       ...
performance  precision        sex_ethnicity training    48.82
             recall           age           training    49.34
                              ethnicity     training    50.65
                              sex           training    50.95
                              sex_ethnicity training    50.16

[128 rows x 1 columns]

In [6]:
audit_result['age']['fairness']

{'ref': 80,
 'training_DI': 98.0,
 'training_SPD': 98.0,
 'training_TPR': 96.82,
 'training_FPR': 99.76,
 'training_PPV': 96.82,
 'training_PNV': 99.76,
 'impact_DI': 99.0,
 'impact_SPD': 98.0}

In [2]:
model.training_results['dxa_inconsistency']

{'sex': {'correlation': 0.0109,
  'proportion': 0.6,
  'rate': 0.0109,
  'normalized_risk': 99.128,
  'bias_level': 'Low/no bias'},
 'ethnicity': {'correlation': 0.0045,
  'proportion': 0.6,
  'rate': 0.0045,
  'normalized_risk': 99.64,
  'bias_level': 'Low/no bias'},
 'age': {'correlation': 0.009,
  'proportion': 0.5522,
  'rate': 0.009,
  'normalized_risk': 99.28,
  'bias_level': 'Low/no bias'},
 'sex_ethnicity': {'correlation': 0.0043,
  'proportion': 0.75,
  'rate': 0.0043,
  'normalized_risk': 99.656,
  'bias_level': 'Low/no bias'}}